In [1]:
%reload_ext watermark
%reload_ext autoreload
%autoreload 2
%watermark -p numpy,sklearn,pandas
%watermark -p cv2,PIL,matplotlib
%watermark -p torch,torchvision,torchaudio
%watermark -p tensorflow,tensorboard
%matplotlib inline
%config InlineBackend.figure_format='retina'
%config IPCompleter.use_jedi = False

from IPython.display import display, Markdown, HTML, Image, Javascript
from IPython.core.magic import register_line_cell_magic, register_line_magic, register_cell_magic
display(HTML('<style>.container { width:%d%% !important; }</style>' % 80))

import sys, os, io, time, random, math
import json, base64, requests
import os.path as osp
import numpy as np

def _IMPORT_(x):
    try:
        segs = x.split(' ')
        g = globals()
        if 'github.com' in segs[1]:
            uri = segs[1].replace('github.com', 'raw.githubusercontent.com')
            mod = uri.split('/')
            for s in ['main', 'master']:
                uri = 'https://' + '/'.join(mod[:-1]) + '/main/' + mod[-1] + '.py'
                x = requests.get(uri)
                if x.status_code == 200:
                    x = x.text
                    break
        elif 'gitee.com' in segs[1]:
            mod = segs[1].split('/')
            for s in ['/raw/main/', '/raw/master/']:
                uri = 'https://' + '/'.join(mod[:3]) + s + '/'.join(mod[3:]) + '.py'
                x = requests.get(uri)
                if x.status_code == 200:
                    x = x.text
                    break
        elif segs[1][0] == '/':
            with open(segs[1] + '.py') as fr:
                x = fr.read()
        exec(x, g)
    except:
        pass


numpy 1.19.5
sklearn not installed
pandas 1.1.5
cv2 4.5.3
PIL 8.3.1
matplotlib 3.3.4
torch not installed
torchvision not installed
torchaudio not installed
tensorflow 2.6.0
tensorboard 2.6.0


In [2]:
import tensorflow as tf

In [3]:
import sys

In [4]:
sys.path.append('/data/nb_data/tf_trt_models')

In [5]:
from PIL import Image
import sys
import os
import urllib
import tensorflow as tf
# import tensorflow.contrib.tensorrt as trt
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import numpy as np
from tf_trt_models.classification import download_classification_checkpoint, build_classification_graph

ModuleNotFoundError: No module named 'nets'

In [9]:
a = np.random.randn(2, 3, 4)
a

array([[[ 0.58821943, -1.50195305, -0.90961352,  1.75272841],
        [ 0.46158129,  1.84278592, -0.09306837,  0.88936903],
        [ 1.06113171,  0.39046257, -0.81616548,  1.35797164]],

       [[-0.12363995, -0.22676362, -0.26167725, -0.72338734],
        [ 0.71351646,  0.17375196, -0.42233735, -1.60346074],
        [-1.66129064,  0.5178923 , -1.34733753, -0.87103162]]])

In [10]:
(a - 0.2) / 0.4

array([[[ 0.97054858, -4.25488263, -2.77403381,  3.88182103],
        [ 0.65395322,  4.10696481, -0.73267092,  1.72342259],
        [ 2.15282928,  0.47615642, -2.5404137 ,  2.89492911]],

       [[-0.80909987, -1.06690906, -1.15419312, -2.30846834],
        [ 1.28379115, -0.06562009, -1.55584336, -4.50865186],
        [-4.65322661,  0.79473075, -3.86834382, -2.67757905]]])

In [11]:
for i in range(2):
    a[i, :, :] = (a[i] - 0.2) / 0.4
a

array([[[ 0.97054858, -4.25488263, -2.77403381,  3.88182103],
        [ 0.65395322,  4.10696481, -0.73267092,  1.72342259],
        [ 2.15282928,  0.47615642, -2.5404137 ,  2.89492911]],

       [[-0.80909987, -1.06690906, -1.15419312, -2.30846834],
        [ 1.28379115, -0.06562009, -1.55584336, -4.50865186],
        [-4.65322661,  0.79473075, -3.86834382, -2.67757905]]])